In [45]:
import os
import cv2
import numpy as np
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers

In [46]:
data_path='D:/Course/Deep Learning/data'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

In [47]:
def load_split(data_Path,categories):

    data = []
    labels = []
    img_size=100
   

    for category in categories:
        folder_path=os.path.join(data_path,category)
        img_names=os.listdir(folder_path)

        for img_name in img_names:
            img_path=os.path.join(folder_path,img_name)
            img=cv2.imread(img_path)
            
            
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)       
            resized=cv2.resize(gray,(img_size,img_size))
               
            data.append(resized)
            labels.append(label_dict[category])
               
     
    
    data=np.array(data)/255.0
    data=np.reshape(data,(data.shape[0],img_size,img_size,1))
    labels = np.array(labels)
    
    return (data, labels)


In [48]:
data,labels=load_split('D:/Course/Deep Learning/data',categories)

In [49]:
from keras.utils import np_utils

new_labels=np_utils.to_categorical(labels)

In [50]:
cnn = models.Sequential([
    layers.Conv2D(filters=200, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 1)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=100, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
 
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])


In [51]:
cnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 98, 98, 200)       2000      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 49, 49, 200)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 47, 47, 100)       180100    
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 23, 23, 100)      0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 52900)             0         
                                                                 
 dense_6 (Dense)             (None, 64)               

In [52]:
cnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [53]:
from sklearn.model_selection import train_test_split

trainX,testX,trainY,testY=train_test_split(data,new_labels,test_size=0.3)

In [54]:
cnn.fit(trainX, trainY, epochs=5)

Epoch 1/5
166/166 [==============================] - 114s 682ms/step - loss: 0.6505 - accuracy: 0.6266
Epoch 2/5
166/166 [==============================] - 111s 671ms/step - loss: 0.5179 - accuracy: 0.7430
Epoch 3/5
166/166 [==============================] - 114s 685ms/step - loss: 0.3623 - accuracy: 0.8368
Epoch 4/5
166/166 [==============================] - 110s 665ms/step - loss: 0.2573 - accuracy: 0.8922
Epoch 5/5
166/166 [==============================] - 111s 667ms/step - loss: 0.1890 - accuracy: 0.9260


In [55]:
cnn.evaluate(testX,testY)


71/71 [==============================] - 12s 162ms/step - loss: 0.3520 - accuracy: 0.8654


[0.3520066440105438, 0.865401566028595]

In [60]:
face_clsfr=cv2.CascadeClassifier("D:/Course/Deep Learning/haarcascade_frontalface_default.xml")

source=cv2.VideoCapture(0)

labels_dict={0:'NO MASK',1:'MASK'}
color_dict={0:(0,0,255),1:(0,255,0)}

In [24]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [61]:
from skimage import io
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(img,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=cnn.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('img',img)
    key=cv2.waitKey(30) & 0xff
    
    if(key==27):
        break
        
source.release()        
cv2.destroyAllWindows()
